In [195]:
import numpy as np
import cv2 as cv


def pre_pro(pic_name, x0, y0):
    # 打开图片
    img = cv.imread(pic_name)
    # 灰度化
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # 二值化
    # 此时的第二个和第三个参数，
    # 即二值化的上下阈值需要根据图片的实际情况个来调整
    ret, binary_img = cv.threshold(gray_img, 180, 255, cv.THRESH_BINARY)
    print("binary_img")
    print(binary_img)
    print(binary_img.shape)
    cv.imshow('binary_img', binary_img)
    cv.waitKey(0)
    cv.destroyAllWindows()

    # 图像像素按行和列求和
    # 二值图像反相
    con_bi_img = 255 - binary_img
    # print(np.shape(binary_img))
    cv.imshow('con_bi_img', con_bi_img)
    cv.waitKey(0)
    cv.destroyAllWindows()
    column_sum_img = np.sum(con_bi_img, axis=0)
    row_sum_img = np.sum(con_bi_img, axis=1)
    # 排序
    sort_column_sum = np.sort(column_sum_img)
    sort_column_sum_indices = np.argsort(column_sum_img)
    sort_row_sum = np.sort(row_sum_img)
    sort_row_sum_indices = np.argsort(row_sum_img)
    print("列：\n")
    print(sort_column_sum[len(sort_column_sum) - 5:])
    print(sort_column_sum_indices[len(sort_column_sum_indices) - 5:])
    print("行：\n")
    print(sort_row_sum[len(sort_row_sum) - 5:])
    print(sort_row_sum_indices[len(sort_row_sum_indices) - 5:])

    # 通过每一列和每一行像素和的输出值，判断坐标轴的位置
    # 在坐标轴范围内遍历每一列，将曲线下面的部分设置成黑色
    for i in range(x0, binary_img.shape[1]):
        flag = 0
        for j in range(0, binary_img.shape[0]):
            if binary_img[j][i] == 0:
                flag += j
                break
        for j in range(flag, y0):
            binary_img[j][i] = 0
    cv.imshow('binary_img', binary_img)
    cv.waitKey(0)
    cv.destroyAllWindows()
    return binary_img

In [196]:
import numpy as np
from numpy.core.fromnumeric import shape
import cv2 as cv

def get_points(picture_name, xMax, xMin, yMax, yMin, x0, y0):

    binary_img = pre_pro(picture_name, x0, y0)
    cv.imshow('binary_img', binary_img)
    cv.waitKey(0)
    cv.destroyAllWindows()

    # 边缘提取
    xgrd = cv.Sobel(binary_img, cv.CV_16SC1, 1, 0)
    ygrd = cv.Sobel(binary_img, cv.CV_16SC1, 0, 1)

    egde_output = cv.Canny(xgrd, ygrd, x0, y0)
    cv.imshow('canny_edge', egde_output)
    cv.waitKey(0)
    cv.destroyAllWindows()

    # 图像像素按行和列求和
    column_sum_img = np.sum(egde_output, axis=0)
    row_sum_img = np.sum(egde_output, axis=1)
    # 排序
    sort_column_sum = np.sort(column_sum_img)
    sort_column_sum_indices = np.argsort(column_sum_img)
    sort_row_sum = np.sort(row_sum_img)
    sort_row_sum_indices = np.argsort(row_sum_img)
    print(sort_column_sum[len(sort_column_sum) - 10:])
    print(sort_column_sum_indices[len(sort_column_sum_indices) - 10:])
    print(sort_row_sum[len(sort_row_sum) - 10:])
    print(sort_row_sum_indices[len(sort_row_sum_indices) - 10:])

    fc = egde_output[0:binary_img.shape[0], 0:binary_img.shape[1]]
    cv.imshow('function_curve', fc)
    cv.waitKey(0)
    cv.destroyAllWindows()

    # 提取图像数据
    rows = (fc.shape)[0]
    cols = (fc.shape)[1]

    min_x = xMin
    max_x = xMax
    min_y = yMin
    max_y = yMax

    x_axis = np.empty([rows, cols])
    y_axis = np.empty([cols, rows])

    # x_interval和y_interval用于调整数据长度，在报错的时候可以通过他们来调整
    x_interval = (max_x - min_x) / (cols + 1)
    x_value = np.arange(min_x + x_interval, max_x, x_interval)
    # 自动调整长度
    if(cols != (x_value.shape)[0]):
        x_interval = (max_x - min_x) / (cols)
        x_value = np.arange(min_x + x_interval, max_x, x_interval)
    y_interval = (max_y - min_y) / (rows + 1)
    y_value = np.arange(max_y - y_interval, min_y, -y_interval)
    if(rows != (y_value.shape)[0]):
        y_interval = (max_y - min_y) / (rows)
        y_value = np.arange(max_y - y_interval, min_y, -y_interval)
    try:
        x_axis[:, ] = x_value
        y_axis[:, ] = y_value
    except:
        max_x +=1
        max_y +=1
        x_axis = np.empty([rows, cols])
        y_axis = np.empty([cols, rows])

        # x_interval和y_interval用于调整数据长度，在报错的时候可以通过他们来调整
        x_interval = (max_x - min_x) / (cols + 1)
        x_value = np.arange(min_x + x_interval, max_x, x_interval)
        # 自动调整长度
        if(cols != (x_value.shape)[0]):
            x_interval = (max_x - min_x) / (cols)
            x_value = np.arange(min_x + x_interval, max_x, x_interval)
        y_interval = (max_y - min_y) / (rows + 1)
        y_value = np.arange(max_y - y_interval, min_y, -y_interval)
        if(rows != (y_value.shape)[0]):
            y_interval = (max_y - min_y) / (rows)
            y_value = np.arange(max_y - y_interval, min_y, -y_interval)
        x_axis[:, ] = x_value
        y_axis[:, ] = y_value   
            
    #excepy 结束
    
    y_axis = y_axis.T

    x_fc = x_axis.T[fc.T == 255]
    y_fc = y_axis.T[fc.T == 255]

    return (x_fc, y_fc)

In [197]:
# generate excel file
import xlsxwriter
import numpy as np 
import os


#函数模板
# pic_name = "demo03(2).png"
# pic_name_original = 'demo03.png'
# res = get_points(pic_name, 2100, 2015, 160, -20, 15, 233)

# pic_name = "15_Extreme weather events - ps-x0-82-y0-42(2).png"
# pic_name_original = '15_Extreme weather events - ps-x0-82-y0-42.png'
# res = get_points(pic_name, 2020, 1880, 150, 60, 0, 0)  #21, 369

# pic_name = "21_A new set of scenarios employed in AR6 - ps - Carbon dioxide - ssp1-1.9(2).png"
# pic_name = f"{pic_name_original.split('.')[0]}(2).{pic_name_original.split('.')[1]}"
# res = get_points(pic_name, 2100, 2015, 140, -20, 0, 0)

# pic_name_original = '24_greenhouse gas emissions scenarios - ps - ssp5-8.5.png'
# res = get_points(pic_name,  2100, 2020, 1.5, 0.1, 0, 0)

# def get_points(picture_name, xMax, xMin, yMax, yMin, x0, y0):

# pic_name_original = 'Image20220715110823.jpg'
# pic_name = f"{os.path.splitext(pic_name_original)[0]}(2).png"
# res = get_points(pic_name,  2021, 1880,4000, 0,  0, 0)

# pic_name_original = '21_A new set of scenarios employed in AR6 - ps - Carbon dioxide - ssp3-7.0.png'
# pic_name = f"{os.path.splitext(pic_name_original)[0]}(2).png"
# res = get_points(pic_name,  2100, 2015,140, -20,  0, 0)

# pic_name_original = '21_A new set of scenarios employed in AR6 - ps - Carbon dioxide - ssp5-8.5.jpg'
# pic_name = f"{os.path.splitext(pic_name_original)[0]}(2).png"
# res = get_points(pic_name,  2101, 2015,140, -20,  0, 0)

# pic_name_original = '21_A new set of scenarios employed in AR6 - ps - Nitrous oxide - SSP5-8.5.png'


pic_name_original = '21_A new set of scenarios employed in AR6 - ps - one air pollutant and contributor to aerosols sulfur dioxide-SSP1-1.9.png'
pic_name = f"{os.path.splitext(pic_name_original)[0]}(2).png"
res = get_points(pic_name,  2100, 2015, 120, 0,  0, 0)


book = xlsxwriter.Workbook(f"{os.path.splitext(pic_name_original)[0]}.xlsx")
sheet = book.add_worksheet(name='data');
sheet.write(0, 0, "x_axis");
sheet.write(0, 1, "y_axis");

xValue = -999999;
row = 1;
points = np.array(res);
[rows, cols] = points.shape
print(points.shape)
print(points)
for j in range(cols):
  if np.floor(points[0][j]) > xValue:
#     print(np.floor(points[0][j]))
    sheet.write(row, 0, np.floor(points[0][j]));
    sheet.write(row, 1, points[1][j]);
    xValue = np.floor(points[0][j]);
    row += 1;

sheet = book.add_worksheet(name = 'picture');
sheet.insert_image('A1',pic_name);
sheet = book.add_worksheet(name = 'picture_original');
sheet.insert_image('A1',pic_name_original);
book.close();

binary_img
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
(80, 158)
列：

[ 765  765  765 1020 1275]
[30 38 17 19 18]
行：

[3825 3825 4080 5355 7395]
[68 65 72 73 66]
[ 765  765  765  765  765  765  765  765  765 1020]
[33 31 17 18 20 29 28 26 23 19]
[1785 2040 2040 2805 2805 3825 4080 5610 7650 7905]
[70 68 63 69 71 64 72 66 67 65]
(2, 269)
[[2015.53797468 2015.53797468 2016.07594937 2016.07594937 2016.61392405
  2016.61392405 2017.15189873 2017.68987342 2017.68987342 2018.2278481
  2018.2278481  2018.76582278 2018.76582278 2019.30379747 2019.84177215
  2019.84177215 2020.37974684 2020.37974684 2020.91772152 2020.91772152
  2021.4556962  2021.4556962  2021.99367089 2022.53164557 2023.06962025
  2023.06962025 2023.60759494 2023.60759494 2024.14556962 2024.14556962
  2024.6835443  2024.6835443  2024.6835443  2025.22151899 2025.22151899
  2025.2215

In [1]:
print(1)

1
